In [1]:
from langgraph.graph import StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Load environment
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key or not api_key.startswith("sk-"):
    raise ValueError("❌ Invalid OpenAI API key. Must start with 'sk-'")
print("API key loaded correctly ✅")

API key loaded correctly ✅


In [2]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5
)
print("LLM initialized ✅")

LLM initialized ✅


In [ ]:
SYSTEM_PROMPT = SystemMessage(
    content=(
        "You are a helpful, polite customer support representative. "
        "You remember what the user previously said and refer to it naturally. "
        "Do not say you lack context if the user already provided it."
    )
)

# Node
def customer_support_node(state: MessagesState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": messages + [response]}

# Graph
graph = StateGraph(MessagesState)
graph.add_node("support_agent", customer_support_node)
graph.set_entry_point("support_agent")
graph.set_finish_point("support_agent")

# Memory
app = graph.compile(checkpointer=MemorySaver())
print("LangGraph app ready ✅")

LangGraph app ready ✅


In [4]:
# Test (Multi-turn)
thread_id = "customer-1"

result1 = app.invoke(
    {
        "messages": [
            SYSTEM_PROMPT,
            HumanMessage(content="I tasted jollof rice for the first time today")
        ]
    },
    config={"configurable": {"thread_id": thread_id}}
)
print("Agent:", result1["messages"][-1].content)

result2 = app.invoke(
    {
        "messages": [
            HumanMessage(content="It's quite spicy")
        ]
    },
    config={"configurable": {"thread_id": thread_id}}
)
print("Agent:", result2["messages"][-1].content)

result3 = app.invoke(
    {
        "messages": [
            HumanMessage(content="Do you remember what I tasted?")
        ]
    },
    config={"configurable": {"thread_id": thread_id}}
)
print("Agent:", result3["messages"][-1].content)


Agent: That's great to hear! Jollof rice is such a flavorful dish. What did you think of it? Did you enjoy the spices and the way it was prepared?
Agent: It can definitely have a kick! The spice level can vary depending on the recipe, but that’s part of what makes jollof rice so delicious. Did you have it with any sides or proteins?
Agent: Yes, you mentioned that you tasted jollof rice for the first time! It sounds like it made quite an impression with its spiciness. Are you thinking about trying it again or maybe exploring other dishes?
